Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
TODAY = '2024-10-18'
NEXTDAY = '2024-10-19'
# PHDAY = '2024-10-16' # place holder

test_split = (TODAY, TODAY)
pos_txt = f'''\
date {TODAY}
cash 100000.00
SH601899 200
SH601816 600
SH601398 500
SH600900 100
'''


EXP_NAME, rid = 'EXP_BENCH', '100ec16fd4bc42d88e64516e9f14c614' # BENCH A checkpoint 
# EXP_NAME, rid = 'EXP_BENCH', 'fb2194b490b94160ab193204ef4bdd4c'
TOPK = 10
NDROP = 2
HT = 10

info = {
    'ALGO': ['GBDT'],
    'market': ['scsi300'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha360"], 
    'label': ['r1'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

Daily data update

In [2]:
s=f'/data/linaiqi/anaconda3/envs/yahoo10/bin/python ~/finance/quniverse/LiLab/lilab/qlib/scripts/data_collector/yahoo/collector.py update_data_to_bin --qlib_data_1d_dir ~/.qlib/qlib_data/cn_data --end_date {NEXTDAY} --delay 0.0 --index_filter csi300_main --index_start_time 2024-09-01 --index_end_time 2024-09-10'
print(s)

/data/linaiqi/anaconda3/envs/yahoo10/bin/python ~/finance/quniverse/LiLab/lilab/qlib/scripts/data_collector/yahoo/collector.py update_data_to_bin --qlib_data_1d_dir ~/.qlib/qlib_data/cn_data --end_date 2024-10-19 --delay 0.0 --index_filter csi300_main --index_start_time 2024-09-01 --index_end_time 2024-09-10


In [3]:
# import os
# os.system(s)

In [4]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/data/linaiqi/anaconda3/envs/qlib37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[3566709:MainThread](2024-10-19 14:22:40,654) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[3566709:MainThread](2024-10-19 14:22:40,657) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[3566709:MainThread](2024-10-19 14:22:40,657) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linaiqi/.qlib/qlib_data/cn_data')}


Record position history

In [5]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
print(dt)
print(account)
print(norm_pos)

2024-10-18 09:00:00
{'SH601899': {'amount': 200}, 'SH601816': {'amount': 600}, 'SH601398': {'amount': 500}, 'SH600900': {'amount': 100}, 'cash': 100000.0}
{'_settle_type': 'None', 'position': {'SH600900': {'amount': 219, 'price': 12.952594757080078}, 'SH601398': {'amount': 827, 'price': 3.8165283203125}, 'SH601816': {'amount': 4001, 'price': 0.8532374501228333}, 'SH601899': {'amount': 1362, 'price': 2.5881271362304688}, 'cash': 100000.0, 'now_account_value': 112931.71937018633}, 'init_cash': 100000.0}


Prepare dataset

In [6]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [7]:
dataset = init_instance_by_config(benchmark.dataset_config)
dataset.prepare('test')

[3566709:MainThread](2024-10-19 14:23:13,368) INFO - qlib.timer - [log.py:127] - Time cost: 32.141s | Loading data Done
[3566709:MainThread](2024-10-19 14:23:14,186) INFO - qlib.timer - [log.py:127] - Time cost: 0.816s | ProcessInf Done
[3566709:MainThread](2024-10-19 14:23:14,247) INFO - qlib.timer - [log.py:127] - Time cost: 0.059s | ZScoreNorm Done
[3566709:MainThread](2024-10-19 14:23:14,250) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | Fillna Done
[3566709:MainThread](2024-10-19 14:23:14,252) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done
[3566709:MainThread](2024-10-19 14:23:14,258) INFO - qlib.timer - [log.py:127] - Time cost: 0.005s | CSZScoreNorm Done
[3566709:MainThread](2024-10-19 14:23:14,258) INFO - qlib.timer - [log.py:127] - Time cost: 0.889s | fit & process data Done
[3566709:MainThread](2024-10-19 14:23:14,259) INFO - qlib.timer - [log.py:127] - Time cost: 33.032s | Init data Done


CLOSE59   CLOSE58   CLOSE57   CLOSE56   CLOSE55  \
datetime   instrument                                                     
2024-10-18 SH600000   -0.599788 -0.677294 -0.778056 -0.588439 -0.548912   
           SH600009    0.572972  0.697194  0.792421  0.857364  0.798274   
           SH600010   -0.440878 -0.429478 -0.445973 -0.369328 -0.303220   
           SH600011    2.501316  2.526639  2.312111  2.241394  2.320451   
           SH600015   -0.419669 -0.482115 -0.600744 -0.441668 -0.361663   
...                         ...       ...       ...       ...       ...   
           SZ002841   -0.879055 -0.827975 -0.911946 -1.048481 -1.093296   
           SZ002916    1.217645  1.067768  1.087139  0.898050  0.727855   
           SZ002920   -0.635769 -0.836312 -0.947000 -1.109048 -1.232034   
           SZ002938    1.133152  1.006846  1.168059  1.096639  0.988702   
           SZ003816    2.037021  1.982912  2.004610  2.238601  2.722747   

                        CLOSE54   CLOSE53   CLOSE52   CLOSE51   CLOSE50  ...  \
datetime   instrument                                                    ...   
2024-10-18 SH600000   -0.568015 -0.805459 -0.576872 -0.679744 -0.916041  ...   
           SH600009    0.976988  0.905699  0.644479  0.690222  0.724280  ...   
           SH600010   -0.340638 -0.288421 -0.243423 -0.146811 -0.243295  ...   
           SH600011    2.418020  2.412705  2.531178  2.489065  2.047476  ...   
           SH600015   -0.491485 -0.654871 -0.434304 -0.453336 -0.673912  ...   
...                         ...       ...       ...       ...       ...  ...   
           SZ002841   -1.120433 -0.997466 -0.947353 -0.843721 -0.933830  ...   
           SZ002916    0.643445  0.651912  0.741985  0.707959  0.586502  ...   
           SZ002920   -1.349683 -1.294985 -1.201843 -1.229068 -1.154681  ...   
           SZ002938    1.019688  1.148504  1.294410  1.166265  1.448128  ...   
           SZ003816    2.861702  2.615869  2.629632  2.512693  1.785929  ...   

                        VOLUME8   VOLUME7   VOLUME6   VOLUME5   VOLUME4  \
datetime   instrument                                                     
2024-10-18 SH600000    0.982379  0.509464  0.711373  0.143310  1.036824   
           SH600009    1.765672  1.046069 -0.215094 -0.244252 -0.240702   
           SH600010    0.606060  0.321645 -0.228732 -0.284738 -0.974338   
           SH600011    0.214372 -0.712901 -0.536786 -0.318167  1.156069   
           SH600015    0.506352  1.336838  0.827817  0.931192  0.880027   
...                         ...       ...       ...       ...       ...   
           SZ002841   -0.686234 -0.966124 -0.803483 -1.068961 -1.266304   
           SZ002916   -1.268426 -1.621672 -1.243155 -0.696353 -1.020407   
           SZ002920   -1.274503 -0.485171 -0.299794  0.271794 -0.054246   
           SZ002938   -0.875147 -2.031476 -1.152238 -1.077946 -0.850938   
           SZ003816    0.441438 -0.230722  0.624096  0.151863 -0.341451   

                        VOLUME3   VOLUME2  VOLUME1  VOLUME0  LABEL0  
datetime   instrument                                                
2024-10-18 SH600000    0.827686  0.490777      0.0      1.0     0.0  
           SH600009   -0.708767 -0.523992      0.0      1.0     0.0  
           SH600010   -0.414753  0.024762      0.0      1.0     0.0  
           SH600011    0.890020 -0.611807      0.0      1.0     0.0  
           SH600015    1.047822  1.194679      0.0      1.0     0.0  
...                         ...       ...      ...      ...     ...  
           SZ002841   -1.577220 -1.266361      0.0      1.0     0.0  
           SZ002916   -0.850983 -0.230904      0.0      1.0     0.0  
           SZ002920   -0.424196 -0.292344      0.0      1.0     0.0  
           SZ002938   -0.706099  1.200547      0.0      1.0     0.0  
           SZ003816   -0.035710 -0.652559      0.0      1.0     0.0  

[237 rows x 361 columns]

Load and run model

In [8]:
with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
    recorder = R.get_recorder(recorder_id=rid)
    model = recorder.load_object("trained_model")

[3566709:MainThread](2024-10-19 14:23:15,488) INFO - qlib.workflow - [exp.py:258] - Experiment 525 starts running ...
[3566709:MainThread](2024-10-19 14:23:15,637) INFO - qlib.workflow - [recorder.py:341] - Recorder ae5cda598d2a4df8a27b3b1dbc194a64 starts running under Experiment 525 ...
[3566709:MainThread](2024-10-19 14:23:16,860) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


In [9]:
EXP_NAME = 'realworld_position_maker'

# backtest and analysis
with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
    recorder = R.get_recorder()
    ba_rid = R.get_recorder().id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

[3566709:MainThread](2024-10-19 14:23:18,107) INFO - qlib.workflow - [exp.py:258] - Experiment 529 starts running ...
[3566709:MainThread](2024-10-19 14:23:18,199) INFO - qlib.workflow - [recorder.py:341] - Recorder 3f4a1191590d4f5f8434f7404e387e0c starts running under Experiment 529 ...
[3566709:MainThread](2024-10-19 14:23:18,358) INFO - qlib.workflow - [record_temp.py:199] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 529
[3566709:MainThread](2024-10-19 14:23:18,375) INFO - qlib.timer - [log.py:127] - Time cost: 0.011s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-10-18 SH600000    0.002518
           SH600009   -0.027068
           SH600010   -0.008423
           SH600011   -0.003775
           SH600015    0.001395


In [10]:
with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
    recorder = R.get_recorder(recorder_id=ba_rid)
    pred_df = recorder.load_object("pred.pkl")

[3566709:MainThread](2024-10-19 14:23:19,608) INFO - qlib.workflow - [exp.py:258] - Experiment 529 starts running ...
[3566709:MainThread](2024-10-19 14:23:19,711) INFO - qlib.workflow - [recorder.py:341] - Recorder 7c34ec6b010b4fb693d3c0a0e92823c0 starts running under Experiment 529 ...
[3566709:MainThread](2024-10-19 14:23:19,903) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


In [11]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)

Top / bottom predictions

In [12]:
rank_df.head(10).T

instrument,SH601872,SH603259,SH603288,SH601658,SZ000733,SH600745,SH600025,SZ000999,SH600028,SH600600
score,0.091835,0.041612,0.040264,0.039727,0.037426,0.034974,0.034678,0.033184,0.032701,0.032568


In [13]:
rank_df.tail(10).T

instrument,SZ000768,SZ002271,SH603799,SZ002555,SZ000776,SZ002459,SZ002410,SH600030,SH600999,SH601319
score,-0.035285,-0.037636,-0.039726,-0.042757,-0.048211,-0.051132,-0.051432,-0.05354,-0.055713,-0.07068


Strategy preparation

In [14]:
strategy_config = {
    "class": "TopkDropoutStrategy",
    "module_path": "lilab.qlib.strategy.signal_strategy",
    "kwargs": {
        "model": model,
        "dataset": dataset,
        "topk": TOPK,
        "n_drop": NDROP,
        "hold_thresh": HT,
        "signal": pred_df
    },
}

port_analysis_config = {
    "executor": benchmark.executor_config,
    "strategy": strategy_config,
    "backtest": benchmark.backtest_config
}

In [15]:
_norm_pos = norm_pos.__dict__['position']
_norm_pos

{'SH600900': {'amount': 219, 'price': 12.952594757080078},
 'SH601398': {'amount': 827, 'price': 3.8165283203125},
 'SH601816': {'amount': 4001, 'price': 0.8532374501228333},
 'SH601899': {'amount': 1362, 'price': 2.5881271362304688},
 'cash': 100000.0,
 'now_account_value': 112931.71937018633}

In [16]:
from qlib.backtest import get_strategy_executor
trade_strategy, trade_executor = \
    get_strategy_executor(TODAY, TODAY, strategy_config, benchmark.executor_config, account=_norm_pos)
trade_executor.reset(start_time=TODAY, end_time=TODAY)
trade_strategy.reset(level_infra=trade_executor.get_level_infra())

[3566709:MainThread](2024-10-19 14:23:20,067) INFO - qlib.backtest caller - [__init__.py:93] - Create new exchange
[3566709:MainThread](2024-10-19 14:23:22,163) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[3566709:MainThread](2024-10-19 14:23:22,165) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[3566709:MainThread](2024-10-19 14:23:22,287) WARNING - qlib.data - [data.py:666] - load calendar error: freq=day, future=True; return current calendar!
[3566709:MainThread](2024-10-19 14:23:22,293) WARNING - qlib.data - [data.py:669] - You can get future calendar by referring to the following document: https://github.com/microsoft/qlib/blob/main/scripts/data_collector/contrib/README.md
[3566709:MainThread](2024-10-19 14:23:22,311) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7f72303bbb10>


In [17]:
dt_lst = sorted(list(pos_history.keys()))[::-1]
for k, v in _norm_pos.items():
    if not isinstance(v, dict):
        continue
    cnt = 1
    for dt in dt_lst:
        tmp_pos = pos_history[dt]
        if k not in tmp_pos.get_stock_list():
            break
        cnt += 1
    print(k, cnt)
    trade_strategy.trade_position.update_stock_count(k, 'day', cnt)

SH600900 5
SH601398 5
SH601816 5
SH601899 5


In [18]:
pred_df0 = pred_df.droplevel(0)
# pred_df0
sell_list, buy_list = trade_strategy.generate_trade_decision(pred_score=pred_df0, today=TODAY)

# Sell Order 

In [19]:
df = pd.DataFrame([[order.stock_id, order.amount] for order in sell_list], columns=['id', 'norm_amount']).set_index('id')
stock_list = df.index.values
fields = ['$close', '$factor', '$close/$factor', \
          '100*(Ref($close, -1) / $close - 1)', '100*(Ref($close, -2) / Ref($close, -1) - 1)']
_df = D.features(stock_list, fields, start_time=pd.to_datetime(TODAY)-pd.to_timedelta(5, 'day'), end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close', 'r0', 'r1']
df_sell_hist = _df


In [20]:
fields = ['$close', '$factor', '$close/$factor']
_df = D.features(stock_list, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close']
_df = _df.droplevel(1)
df = pd.concat([df,_df],axis=1,join='inner')
df['amount'] = df['norm_amount'] * df['factor']
df['receive'] = df['amount'] * df['close']
df['amount'] = df['amount'].apply(lambda x: round(x/100)*100)
df = df.loc[df['amount'] > 0, :]
df

,norm_amount,norm_close,factor,close,amount,receive


In [21]:
print(df['receive'].sum())

0


# Buy Order

In [22]:
df = pd.DataFrame([[order.stock_id, order.amount] for order in buy_list], columns=['id', 'norm_amount']).set_index('id')
stock_list = df.index.values
fields = ['$close', '$factor', '$close/$factor', \
          '100*(Ref($close, -1) / $close - 1)', '100*(Ref($close, -2) / Ref($close, -1) - 1)']
_df = D.features(stock_list, fields, start_time=pd.to_datetime(TODAY)-pd.to_timedelta(5, 'day'), end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close', 'r0', 'r1']
df_buy_hist = _df

In [23]:
fields = ['$close', '$factor', '$close/$factor']
_df = D.features(stock_list, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close']
_df = _df.droplevel(1)
df = pd.concat([df,_df],axis=1,join='inner')
df['amount'] = df['norm_amount'] * df['factor']
df['pay'] = df['amount'] * df['close']
df['amount'] = df['amount'].apply(lambda x: round(x/100)*100)
df = df.loc[df['amount'] > 0, :]
df

,norm_amount,norm_close,factor,close,amount,pay
SH601872,7842.061804,1.510317,0.178524,8.460001,1400,11844.001389
SH603259,2200.162581,2.739070,0.045451,60.264004,100,6026.400375
SH603288,1673.797069,4.485610,0.059744,75.080017,100,7508.001709
SH601658,10832.269304,1.093400,0.193865,5.640000,2100,11843.999720
SZ000733,1976.673008,4.744336,0.101180,46.889999,200,9377.999878
SH600745,2580.675078,3.802493,0.116249,32.709999,300,9812.999725
SH600025,4833.340551,2.375996,0.227586,10.440000,1100,11483.999538
SZ000999,3158.154474,3.751621,0.063328,59.241001,200,11848.200226


In [24]:
print(df['pay'].sum())

79745.60256004333


Check

In [25]:
df_sell_hist

,,norm_close,factor,close,r0,r1
instrument,datetime,,,,,


In [26]:
df_buy_hist

norm_close    factor      close        r0        r1
instrument datetime                                                       
SH600025   2024-10-14    2.416962  0.227586  10.619999 -2.071559  0.961542
           2024-10-15    2.366893  0.227586  10.399999  0.961542       NaN
           2024-10-16    2.389652  0.227586  10.500000       NaN       NaN
           2024-10-17         NaN  0.227586        NaN       NaN       NaN
           2024-10-18    2.375996  0.227586  10.440000       NaN       NaN
SH600745   2024-10-14    3.699032  0.116249  31.820002 -1.822758 -1.792574
           2024-10-15    3.631608  0.116249  31.240002 -1.792574       NaN
           2024-10-16    3.566509  0.116249  30.680000       NaN       NaN
           2024-10-17         NaN  0.116249        NaN       NaN       NaN
           2024-10-18    3.802493  0.116249  32.709999       NaN       NaN
SH601658   2024-10-14    1.093400  0.193865   5.640000 -2.836883  3.467166
           2024-10-15    1.062381  0.193865   5.480000  3.467166       NaN
           2024-10-16    1.099216  0.193865   5.670000       NaN       NaN
           2024-10-17         NaN  0.193865        NaN       NaN       NaN
           2024-10-18    1.093400  0.193865   5.640000       NaN       NaN
SH601872   2024-10-14    1.608863  0.178524   9.012000 -5.725706  0.141251
           2024-10-15    1.516744  0.178524   8.495999  0.141251       NaN
           2024-10-16    1.518886  0.178524   8.508000       NaN       NaN
           2024-10-17         NaN  0.178524        NaN       NaN       NaN
           2024-10-18    1.510317  0.178524   8.460001       NaN       NaN
SH603259   2024-10-14    2.830700  0.045451  62.280006 -3.506744 -2.236432
           2024-10-15    2.731435  0.045451  60.096008 -2.236432       NaN
           2024-10-16    2.670348  0.045451  58.752003       NaN       NaN
           2024-10-17         NaN  0.045451        NaN       NaN       NaN
           2024-10-18    2.739070  0.045451  60.264004       NaN       NaN
SH603288   2024-10-14    4.831218  0.059744  80.864792 -2.283996 -4.762954
           2024-10-15    4.720873  0.059744  79.017838 -4.762954       NaN
           2024-10-16    4.496020  0.059744  75.254257       NaN       NaN
           2024-10-17         NaN  0.059744        NaN       NaN       NaN
           2024-10-18    4.485610  0.059744  75.080017       NaN       NaN
SZ000733   2024-10-14    4.485314  0.101180  44.329998 -2.165580 -0.668651
           2024-10-15    4.388181  0.101180  43.369995 -0.668651       NaN
           2024-10-16    4.358840  0.101180  43.080002       NaN       NaN
           2024-10-17         NaN  0.101180        NaN       NaN       NaN
           2024-10-18    4.744336  0.101180  46.889999       NaN       NaN
SZ000999   2024-10-14    3.865232  0.063328  61.035004 -1.022363 -2.001297
           2024-10-15    3.825716  0.063328  60.411003 -2.001297       NaN
           2024-10-16    3.749152  0.063328  59.202000       NaN       NaN
           2024-10-17         NaN  0.063328        NaN       NaN       NaN
           2024-10-18    3.751621  0.063328  59.241001       NaN       NaN